In [14]:
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

#dados que serão utilizados
url = "https://mantis.saojudas.base2.com.br/login_page.php"

data_inicio = [["2020","1","1"], ["2021","5","1"], ["2021","12","1"]]
data_fim = [["2020","1","30"], ["2021","5","30"], ["2021","12","30"]]

In [15]:
#realizar login e criar tarefa teste
def setup(navegador):
    #realiza login
    navegador.find_element_by_xpath('//*[@id="username"]').send_keys('Fernanda')
    navegador.find_element_by_xpath('//*[@id="login-form"]/fieldset/input[2]').click()
    navegador.find_element_by_xpath('//*[@id="password"]').send_keys(81721367)
    navegador.find_element_by_xpath('//*[@id="login-form"]/fieldset/input[3]').click()
    assert "https://mantis.saojudas.base2.com.br/my_view_page.php" in navegador.current_url
    
    #cria nova tarefa teste (preenchendo apenas resumo e descricao)
#    navegador.find_element_by_xpath('//*[@id="navbar-container"]/div[2]/ul/li[1]/div/a').click()   
#    assert "https://mantis.saojudas.base2.com.br/bug_report_page.php" in navegador.current_url
#    navegador.find_element_by_name('summary').send_keys('resumo tarefa teste')
#    navegador.find_element_by_name('description').send_keys('descricao tarefa teste')
#    navegador.find_element_by_xpath('//*[@id="report_bug_form"]/div/div[2]/div[2]/input').click()
#    WebDriverWait(navegador, 10).until(
#        EC.url_contains("https://mantis.saojudas.base2.com.br/view.php"))

In [16]:
def filtrar_por_atribuicao(): 
    #abre navegador e acessa url
    navegador = webdriver.Chrome()
    navegador.get(url)

    #realiza login
    setup(navegador)

    #acessa página Ver Tarefas
    navegador.find_element_by_xpath('//*[@id="sidebar"]/ul/li[2]/a').click()    
    assert "https://mantis.saojudas.base2.com.br/view_all_bug_page.php" in navegador.current_url

    #redefine filtros
    navegador.find_element_by_xpath('//*[@id="filter"]/div[2]/div/div/div/div/a[1]').click()

    #ativa filtro "Atribuído a"
    navegador.find_element_by_xpath('//*[@id="handler_id_filter"]').click()

    #verifica se o select apareceu
    try:
        WebDriverWait(navegador, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="handler_id_filter_target"]/select'))   
        )
    finally: 
        #carrega select e as options
        select = Select(navegador.find_element_by_xpath('//*[@id="handler_id_filter_target"]/select'))
        options = select.options

    #percorre o select
    for index in range(0, len(options)):

        #seleciona item do select
        select.select_by_index(index)

        #faz a filtragem
        navegador.find_element_by_xpath('//*[@id="filters_form_open"]/div[2]/div/div/input[2]').click()

        #recarrega o select devido à atualização que a página fez após a filtragem
        navegador.find_element_by_xpath('//*[@id="handler_id_filter"]').click()   
        try:
            WebDriverWait(navegador, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="handler_id_filter_target"]/select'))   
            )
        finally:  
            select = Select(navegador.find_element_by_xpath('//*[@id="handler_id_filter_target"]/select'))
            options = select.options
            #guarda texto do item que foi selecionado
            objeto = options[index].text

        #espera carregar tabela de tarefas filtradas
        try:
            WebDriverWait(navegador, 10).until(
                EC.presence_of_element_located((By.ID, 'buglist'))
            ) 
        finally:
            #guarda elementos da tabela
            elements = navegador.find_elements_by_class_name('column-status')
            #deleta coluna "Estado" da lista de elementos
            del elements[0]

            #realiza verificacoes, exceto para tarefas com atribuicao "qualquer"
            if len(elements) > 0 and objeto != "[qualquer]":
                #percorre elementos da tabela
                for e in elements:
                    #verifica se retornou elementos corretamente filtrados pra cada membro do grupo
                    if objeto[0] != "[":
                        element = navegador.find_element_by_xpath('//*[@id="buglist"]/tbody/tr['+str(elements.index(e)+1)+']/td[9]/div/a')
                        assert element.text == objeto
                    #verifica se elementos sem nenhuma atribuicao estao marcados como "novo"
                    elif objeto == "[nenhum]":
                        element = navegador.find_element_by_xpath('//*[@id="buglist"]/tbody/tr['+str(elements.index(e)+1)+']/td[9]/div/span')
                        assert element.text == "novo"
                    #verifica elementos atribuidos ao próprio usuario
                    else:
                        element = navegador.find_element_by_xpath('//*[@id="buglist"]/tbody/tr['+str(elements.index(e)+1)+']/td[9]/div/a')
                        user = navegador.find_element_by_xpath('//*[@id="navbar-container"]/div[2]/ul/li[2]/a/span').text
                        assert element.text == user

    navegador.close()
    print("Filtrar por atribuição - OK")

filtrar_por_atribuicao()

Filtrar por atribuição - OK


In [17]:
def filtrar_por_gravidade(): 
    #abre navegador e acessa url
    navegador = webdriver.Chrome()
    navegador.get(url)

    #realiza login
    setup(navegador)

    #acessa página Ver Tarefas
    navegador.find_element_by_xpath('//*[@id="sidebar"]/ul/li[2]/a').click()    
    assert "https://mantis.saojudas.base2.com.br/view_all_bug_page.php" in navegador.current_url

    #redefine filtros
    navegador.find_element_by_xpath('//*[@id="filter"]/div[2]/div/div/div/div/a[1]').click()

    #ativa filtro "Gravidade"
    navegador.find_element_by_xpath('//*[@id="show_severity_filter"]').click()

    #verifica se o select apareceu
    try:
        WebDriverWait(navegador, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="show_severity_filter_target"]/select'))   
        )
    finally: 
        #carrega select e as options
        select = Select(navegador.find_element_by_xpath('//*[@id="show_severity_filter_target"]/select'))
        options = select.options

    #percorre o select
    for index in range(0, len(options)):

        #seleciona item do select
        select.select_by_index(index)

        #faz a filtragem
        navegador.find_element_by_xpath('//*[@id="filters_form_open"]/div[2]/div/div/input[2]').click()

        #recarrega o select devido à atualização que a página fez após a filtragem
        navegador.find_element_by_xpath('//*[@id="show_severity_filter"]').click()   
        try:
            WebDriverWait(navegador, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="show_severity_filter_target"]/select'))   
            )
        finally:  
            select = Select(navegador.find_element_by_xpath('//*[@id="show_severity_filter_target"]/select'))
            options = select.options
            #guarda texto do item que foi selecionado
            objeto = options[index].text

        #espera carregar tabela de tarefas filtradas
        try:
            WebDriverWait(navegador, 10).until(
                EC.presence_of_element_located((By.ID, 'buglist'))
            ) 
        finally:
            #guarda elementos da tabela
            elements = navegador.find_elements_by_class_name('column-severity')
            #deleta coluna "Gravidade" da lista de elementos
            del elements[0]

            #realiza verificacoes, exceto para tarefas com gravidade "qualquer"
            if len(elements) > 0 and objeto != "[qualquer]":
                #percorre elementos da tabela
                for e in elements:
                    #verifica se retornou elementos corretamente filtrados
                    element = navegador.find_element_by_xpath('//*[@id="buglist"]/tbody/tr['+str(elements.index(e)+1)+']/td[8]')
                    assert element.text == objeto

    navegador.close()
    print("Filtrar por gravidade - OK")
    
filtrar_por_gravidade()

Filtrar por gravidade - OK


In [18]:
def filtrar_por_estado():     
    #abre navegador e acessa url
    navegador = webdriver.Chrome()
    navegador.get(url)

    #realiza login
    setup(navegador)

    #acessa página Ver Tarefas
    navegador.find_element_by_xpath('//*[@id="sidebar"]/ul/li[2]/a').click()    
    assert "https://mantis.saojudas.base2.com.br/view_all_bug_page.php" in navegador.current_url

    #redefine filtros
    navegador.find_element_by_xpath('//*[@id="filter"]/div[2]/div/div/div/div/a[1]').click()

    #ativa filtro "Estado"
    navegador.find_element_by_xpath('//*[@id="show_status_filter"]').click()

    #verifica se o select apareceu
    try:
        WebDriverWait(navegador, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="show_status_filter_target"]/select'))   
        )
    finally: 
        #carrega select e as options
        select = Select(navegador.find_element_by_xpath('//*[@id="show_status_filter_target"]/select'))
        options = select.options

    #percorre o select
    for index in range(0, len(options)):

        #seleciona item do select
        select.select_by_index(index)

        #faz a filtragem
        navegador.find_element_by_xpath('//*[@id="filters_form_open"]/div[2]/div/div/input[2]').click()

        #recarrega o select devido à atualização que a página fez após a filtragem
        navegador.find_element_by_xpath('//*[@id="show_status_filter"]').click()   
        try:
            WebDriverWait(navegador, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="show_status_filter_target"]/select'))   
            )
        finally:  
            select = Select(navegador.find_element_by_xpath('//*[@id="show_status_filter_target"]/select'))
            options = select.options
            #guarda texto do item que foi selecionado
            objeto = options[index].text

        #espera carregar tabela de tarefas filtradas
        try:
            WebDriverWait(navegador, 10).until(
                EC.presence_of_element_located((By.ID, 'buglist'))
            ) 
        finally:
            #guarda elementos da tabela
            elements = navegador.find_elements_by_class_name('column-status')
            #deleta coluna "Gravidade" da lista de elementos
            del elements[0]

            #realiza verificacoes, exceto para tarefas com estado "qualquer"
            if len(elements) > 0 and objeto != "[qualquer]":
                #percorre elementos da tabela
                for e in elements:
                    #verifica se retornou elementos corretamente filtrados
                    element = navegador.find_element_by_xpath('//*[@id="buglist"]/tbody/tr['+str(elements.index(e)+1)+']/td[9]/div/span')
                    assert element.text == objeto

    navegador.close()
    print("Filtrar por estado - OK")

filtrar_por_estado()

Filtrar por estado - OK


In [19]:
def filtrar_por_data_alteracao():
    #abre navegador e acessa url
    navegador = webdriver.Chrome()
    navegador.get(url)

    #realiza login
    setup(navegador)

    #acessa página Ver Tarefas
    navegador.find_element_by_xpath('//*[@id="sidebar"]/ul/li[2]/a').click()    
    assert "https://mantis.saojudas.base2.com.br/view_all_bug_page.php" in navegador.current_url

    #redefine filtros
    navegador.find_element_by_xpath('//*[@id="filter"]/div[2]/div/div/div/div/a[1]').click()

    #ativa filtro "Data da ultima alteração"
    navegador.find_element_by_xpath('//*[@id="do_filter_by_last_updated_date_filter"]').click()

    #seleciona checkbox
    try:
        WebDriverWait(navegador, 10).until(
            EC.visibility_of_element_located((By.XPATH, '//*[@id="do_filter_by_last_updated_date_filter_target"]/table/tbody/tr[2]/td[2]/select[1]'))   
        )
    finally:
        navegador.find_element_by_css_selector('#do_filter_by_last_updated_date_filter_target > table > tbody > tr:nth-child(1) > td > label > span').click()
        
    #pesquisa cada data de teste     
    for i in range(0, len(data_inicio)):
        #verifica se o select apareceu
        try:
            WebDriverWait(navegador, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="do_filter_by_last_updated_date_filter_target"]/table/tbody/tr[2]/td[2]/select[1]'))   
            )
        finally: 
            #define datas em cada select
            ano_inicio = Select(navegador.find_element_by_xpath('//*[@id="do_filter_by_last_updated_date_filter_target"]/table/tbody/tr[2]/td[2]/select[1]'))
            ano_inicio.select_by_visible_text(data_inicio[i][0])
            mes_inicio = Select(navegador.find_element_by_xpath('//*[@id="do_filter_by_last_updated_date_filter_target"]/table/tbody/tr[2]/td[2]/select[2]'))
            mes_inicio.select_by_value(data_inicio[i][1])
            dia_inicio = Select(navegador.find_element_by_xpath('//*[@id="do_filter_by_last_updated_date_filter_target"]/table/tbody/tr[2]/td[2]/select[3]'))
            dia_inicio.select_by_visible_text(data_inicio[i][2])
            
            #monta data_inicio e transforma em datetime
            d_inicio = data_inicio[i][0] + "-" + data_inicio[i][1]+ "-" + data_inicio[i][2]
            d_inicio = datetime.strptime(d_inicio, '%Y-%m-%d').date()
                        
            ano_fim = Select(navegador.find_element_by_xpath('//*[@id="do_filter_by_last_updated_date_filter_target"]/table/tbody/tr[3]/td[2]/select[1]'))
            ano_fim.select_by_visible_text(data_fim[i][0])
            mes_fim = Select(navegador.find_element_by_xpath('//*[@id="do_filter_by_last_updated_date_filter_target"]/table/tbody/tr[3]/td[2]/select[2]'))
            mes_fim.select_by_value(data_fim[i][1])
            dia_fim = Select(navegador.find_element_by_xpath('//*[@id="do_filter_by_last_updated_date_filter_target"]/table/tbody/tr[3]/td[2]/select[3]'))
            dia_fim.select_by_visible_text(data_fim[i][2])
            
            #monta data_fim e transforma em datetime
            d_fim = data_fim[i][0] + "-" + data_fim[i][1]+ "-" + data_fim[i][2]
            d_fim = datetime.strptime(d_fim, '%Y-%m-%d').date()

            #faz a filtragem
            navegador.find_element_by_xpath('//*[@id="filters_form_open"]/div[2]/div/div/input[2]').click() 
    
            #espera carregar tabela de tarefas filtradas
            try:
                WebDriverWait(navegador, 10).until(
                    EC.presence_of_element_located((By.ID, 'buglist'))
                ) 
            finally:
                #guarda elementos da tabela
                elements = navegador.find_elements_by_class_name('column-last-modified')
                #deleta coluna "Atualizado" da lista de elementos
                del elements[0]   

                #realiza verificacoes, exceto para tarefas com estado "qualquer"
                if len(elements) > 0:
                    #percorre elementos da tabela
                    for e in elements:
                        #converte data do elemento para datetime
                        date = datetime.strptime(e.text, '%Y-%m-%d').date()
                        
                        #verifica se retornou elementos corretamente filtrados
                        assert date >= d_inicio and date <= d_fim
                        
            #reativa filtro "Data da ultima alteração"
            navegador.find_element_by_xpath('//*[@id="do_filter_by_last_updated_date_filter"]').click()
    
    navegador.close()
    print("Filtrar por Data da Última Alteração - OK")
    
filtrar_por_data_alteracao()

Filtrar por Data da Última Alteração - OK


In [20]:
def salvar_filtro():
    #abre navegador e acessa url
    navegador = webdriver.Chrome()
    navegador.get(url)

    #realiza login
    setup(navegador)

    #acessa página Ver Tarefas
    navegador.find_element_by_xpath('//*[@id="sidebar"]/ul/li[2]/a').click()    
    assert "https://mantis.saojudas.base2.com.br/view_all_bug_page.php" in navegador.current_url

    #redefine filtros
    navegador.find_element_by_xpath('//*[@id="filter"]/div[2]/div/div/div/div/a[1]').click()

    #ativa filtro "Monitorado por"
    navegador.find_element_by_id('user_monitor_filter').click()

    #verifica se o select apareceu
    try:
        WebDriverWait(navegador, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="user_monitor_filter_target"]/select'))   
        )
    finally: 
        #carrega select e as options
        select = Select(navegador.find_element_by_xpath('//*[@id="user_monitor_filter_target"]/select'))
        select.select_by_visible_text('[Eu mesmo]')
    
    navegador.find_element_by_xpath('//*[@id="filter"]/div[2]/div/div/div/div/a[2]').click()
    
    #verifica se esta na pagina de salvar o filtro
    assert 'https://mantis.saojudas.base2.com.br/query_store_page.php' in navegador.current_url
    
    #nomeia e salva o filtro
    filtro_teste = "filtro teste"
    navegador.find_element_by_xpath('//*[@id="save-filter"]/div[2]/div/form[1]/input[2]').send_keys(filtro_teste)
    navegador.find_element_by_xpath('//*[@id="save-filter"]/div[2]/div/form[1]/input[3]').click()
    
    assert 'https://mantis.saojudas.base2.com.br/view_all_bug_page.php' in navegador.current_url
    
    #carrega opcoes do select em uma lista
    select = Select(navegador.find_element_by_xpath('//*[@id="filter-queries-form"]/label/select'))
    options = select.options
    lista = []
    
    for e in options:
        lista.append(e.text)
            
    #verifica se o filtro criado está na lista
    assert filtro_teste in lista
    
    navegador.close()
    print("Salvar filtro - OK")

salvar_filtro()

Salvar filtro - OK
